In [ ]:
import sys
import os
from src.botEnv import botEnv
import random #nur zum testen
import neat
import pybullet as p
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle


In [ ]:
mode = 1 #0 = nur Positionen, 1 = Orientation
numberAxis = 6 #Anzahl der Achsen 1,2,3,6,7
numPositions = 1000
name = "positions" #Name des Output files

In [ ]:
#Bringt den Roboter in eine zufällige Position, überprüft ob eine Kollision stattgefunden hat. Falls nicht wird die Position zu den möglichen Zielpositionen hinzugefügt.
#gibt ein File mit Pickel zurück, welches die gewünschte Anzahl an Zielpunkten in Form einer Liste aus Listen enthält
#goalPosition[0] = x-Koordinate, [1] = y-Koordinate, [2] = z-Koordinate, ([3] = roll, [4] = pitch, [5] = yaw; falls Mode = 1)


def getJointPos(upperLimits, lowerLimits, moveable):
    numJoints = len(upperLimits)
    jointPos = []
    for index in range(numJoints):
        if moveable[index]:
            pos = random.uniform(lowerLimits[index], upperLimits[index]) 
        else:
            pos = 0    
        jointPos.append(pos)
    return jointPos     






p.connect(p.GUI)
bot = botEnv(startPos = [0,0,0], numAxis = numberAxis)
print(bot.moveable)
num = 0
hit = False
goalPosition = []
while(num<numPositions):
    hit = False
    position = []
    bot.jointPosition = getJointPos(bot.upperLimits, bot.lowerLimits, bot.moveable) #Falls nur bestimmte Achsenbereiche zugelassen werden sollen, hier upper oder lower limit durch einenen Array ersetzen
    for i in range(50):       
        p.stepSimulation()
        time.sleep(1/240)
        for i in range(bot.numJoints):
            if bot.moveable:
                p.setJointMotorControl2(bot.botId, i, p.POSITION_CONTROL, targetPosition=bot.jointPosition[i])
            else:
                p.setJointMotorControl2(bot.botId, i, p.POSITION_CONTROL, targetPosition=0)
        if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.planeId)) > 0: #überprüft Kollsion mit Boden
            hit = True
        if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.botId)) > 0: #überprüft kollision mit sich selbst
            hit = True
    if not hit:
        pos = p.getLinkState(bot.botId, bot.numJoints-1)[0] #pos[0] = x, pos[1] = y, pos[2]=z
        #####Falls nur bestimmte Positionen erwünscht sind, hier eine If Abfrage einbauen
        for i in pos:
            position.append(i)   
        if mode == 1:
            orientationQuat = p.getLinkState(bot.botId, bot.numJoints-1)[1]
            orientation = p.getEulerFromQuaternion(orientationQuat) # [0] = pitch, [1] = yaw, [2] = roll
            for i in orientation:
                position.append(i) 
        num += 1
        print(position)
        goalPosition.append(position) #[0] = x-Koordinate, [1] = y-Koordinate, [2] = z-Koordinate, ([3] = pitch, [4] = yaw, [5] = roll; falls Mode = 1)
    bot.botReset()
p.disconnect()
with open(name, 'wb') as fp:
    pickle.dump(goalPositions, fp)

In [8]:
#Für 1-3 Achsen kann auch diese Funkion verwendet werden, ist schneller und lässt sich besser begrenzen. beruht darauf, dass für den dreiachsigen Fall alle Punkte auf einer Hohlkugel um
#0.36 liegen
name = 'Pos2-Axis'

def getGoalPosition(upperRad, lowerRad, upperTheta, lowerTheta, upperPhi, lowerPhi, num): #Minimum Radius = 0.465, Höhe Gelenk = 0.36, MaxRadius = 0.921
    goalPositions = []
    for i in range(num):
        rad = random.uniform(lowerRad, upperRad)
        theta = random.uniform(lowerTheta, upperTheta)
        phi = random.uniform(lowerPhi, upperPhi)
        #phi = 0 #Auskommentieren für 2 achsigen Fall 
        #r = 0.6
        #theta = 43
        zOffset = 0.36 #Höhe des Gelenkes
        x = math.sin(math.radians(theta)) * math.cos(math.radians(phi)) * rad
        y = math.sin(math.radians(theta)) * math.sin(math.radians(phi)) * rad
        z = math.cos(math.radians(theta)) * rad + zOffset
        goal = [x,y,z]
        goalPositions.append(goal)
    print(goalPositions)

    return goalPositions


goalPositions = getGoalPosition(0.921, 0.465, 110, 0, 169 ,-169 ,1000) #maximaler Radius = 0.921, minimaler Radius = 0.465, theta max =110°, min = 0°, Phi max 180, min -180
with open(name, 'wb') as fp:
    pickle.dump(goalPositions, fp)

[[0.6761523003820049, 0.0, 0.6154020010169694], [0.27326321887833854, 0.0, 0.8661860720266301], [0.4520949306748894, 0.0, 1.0033112735821161], [0.4599999968351132, 0.0, 0.8343953110511085], [0.47758987207198067, 0.0, 0.8357681486658679], [0.8500457505874905, 0.0, 0.6716090510603665], [0.7713908421319976, 0.0, 0.24965545527809668], [0.12077975571915583, 0.0, 0.8718253696391918], [0.023126348119734318, 0.0, 0.9059893781301622], [0.2621681885101961, 0.0, 0.804977220274754], [0.5092847982892206, 0.0, 0.3151875992786284], [0.8599637066355373, 0.0, 0.43547336795085384], [0.48730699843626357, 0.0, 0.2773690705318783], [0.5380439514555518, 0.0, 0.19482552383777166], [0.6736711062329656, 0.0, 0.7246964821027058], [0.6957521132267523, 0.0, 0.9180616131970187], [0.22715441806932177, 0.0, 0.8457773606306231], [0.39548656529683524, 0.0, 0.9651463260452321], [0.46042752808367926, 0.0, 0.9762495753877164], [0.25285388433217876, 0.0, 0.8315354313272474], [0.04964692105939848, 0.0, 1.1711806782954777],